## Benchmark Model

***Note: make sure you have [preprocessed](./ENindoor67-Preprocessing.ipynb) images for `efficientnet-b0`***

Our benchmark model is a fine-tuned [ResNeXt101_32x16d_WSL](https://pytorch.org/hub/facebookresearch_WSL-Images_resnext/) CNN. This model is replicated from the github repo of [ashrutkumar](https://github.com/ashrutkumar/Indoor-scene-recognition) with the the `softmax` layer replaced by `torch.max` in the training script.

In [1]:
import sys
import os
import pickle
import sagemaker
from sagemaker.session import Session

sys.path.append('../source')
session = Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()

Load the metadata

In [2]:
root_dir = '../data/mit_indoor_67/metadata/'
efficientnet = 'efficientnet_b0'  # Image size (3,224, 224)
metadata_file = root_dir + efficientnet + ".pkl"
metadata = pickle.load(open(metadata_file, 'rb'))
metadata

{'test': 's3://sagemaker-us-east-2-194071253362/mit_indoor_67/processed/test/efficientnet_b0/indoor67_test.pkl',
 'train': 's3://sagemaker-us-east-2-194071253362/mit_indoor_67/processed/train/efficientnet_b0',
 'val': 's3://sagemaker-us-east-2-194071253362/mit_indoor_67/processed/val/efficientnet_b0'}

Define output_path, source directory and dependencies

In [3]:
prefix = 'mit_indoor_67'
output_path = os.path.join('s3://', bucket, prefix)
print('model artefacts will be saved to: {}'.format(output_path))

model artefacts will be saved to: s3://sagemaker-us-east-2-194071253362/mit_indoor_67


In [4]:
source_dir = '../source'
dependencies = ['../source/dataset', '../source/utils']

The training script:

In [5]:
!pygmentize ../source/train.py

Error: cannot read infile: [Errno 2] No such file or directory: '../source/train.py'


Set job name

In [6]:
from time import gmtime, strftime
job_name = "mitindoor67-resnext101-32x16d-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
print(job_name)

mitindoor67-resnext101-32x16d-2020-11-26-04-10-32


Set instance details, Pytorch framework version and hyperparameters:

The model uses a `torch.optim.SGD` optimizer with `learning_rate` : 0.001, `momentum` of 0.9, and `subsetrandom` sampling. Early stopping was not implemented in the [original script](https://github.com/ashrutkumar/Indoor-scene-recognition/blob/master/indoor_scene_recognition.ipynb) as it claims and hence we will train the model up to 20 epochs initially; if the model converges, we will take it as the benchmark model.

The model with the best `val_acc` will be saved.

In [7]:
instance_type = 'ml.p3.2xlarge'
instance_count = 1
framework_version = '1.6.0'

hyperparameters = {
                    'model' : 'ResNext101',  # this will load pretrained ResNeXt101-32x16d_WSL
                    'epochs': 20, 
                    'batch-size' : 50,  # batch size - original model HP
                    'sampling' : 'subsetrandom',  # sampling method - original model setting
                    'workers' : 7,  # number of workers
                    'lr' : 0.001,  #  learning rate - original model HP
                    'optimizer' : 'sgd',  # optimizer - original model setting
                    'momentum' : 0.9,  # SGD momentum - original model HP
                    'dropout' : 0.5,  # dropout probability - original model HP
                    'patience' : 20  # We will train up to 20 epochs without early stopping
                    } 


Instantiate the estimator

In [8]:
from sagemaker.pytorch import PyTorch

# initial attempt
estimator = PyTorch(entry_point='main.py',
                    source_dir=source_dir,
                    dependencies=dependencies,
                    role=role,
                    instance_count=instance_count,
                    instance_type=instance_type,
                    framework_version=framework_version,
                    py_version='py3',
                    output_path=output_path,
                    sagemaker_session=session,
                    hyperparameters=hyperparameters
                   )

Fit the training and validation data.

In [9]:
estimator.fit({
    'train': metadata['train'],
    'val' : metadata['val']},
    job_name=job_name,
    wait = False)  # set to `False` then we can attach the model to a new estimator later

Let's now move on to training our [EfficientNet base model](./EfficientNets-Base.ipynb).